In [1]:
import torch
from model import PoseFlowModel
from train import Trainer
from utils.dataset_gen import generate_dataset
from viewer.viewer import Viewer
from viewer.amc_parser import parse_asf
from utils.parser import parse_motion_file
from utils.preprocessing import downsample_motion, tensor_to_motion_frames, motion_frames_to_tensor
from data import metadata

pygame 2.5.2 (SDL 2.28.2, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
batch_size = 64
block_size = 30
frame_rate = 15
dataset_path = "dataset.bin"
checkpoint_path = "checkpoint.pth"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

device = 'cpu'


In [24]:
dataset = generate_dataset('data/subjects/01', block_size, frame_rate).to(torch.float32)
torch.save(dataset, dataset_path)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x718e78c954c0>>
Traceback (most recent call last):
  File "/home/michaseyi/research/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 801, in _clean_thread_parent_frames
    for identity in list(thread_to_parent.keys()):
                         ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


In [3]:
model = PoseFlowModel(block_size=block_size, pose_embd=228, block_embd=176, num_layers=12, num_heads=8, dropout=0.2, device=device).to(device)
model.parameter_count()

4701396

In [4]:
trainer = Trainer(model, dataset_path, checkpoint_path, device=device, early_stopper_patience=30)

Loading checkpoint from checkpoint.pth:
Epoch 803: train loss - 0.019386590899457188, val loss - 0.01945284143628844


In [ ]:
trainer.train()

In [50]:
skeleton = parse_asf('data/subjects/05/05.asf')
motion = parse_motion_file('data/subjects/05/05_02.amc')
motion.frame_rate = metadata['5']['3'].frame_rate
motion = downsample_motion(motion, frame_rate)
motion_tensor = motion_frames_to_tensor(motion.frames)

In [29]:
start_motion = motion_tensor[50: 50 + block_size].clone().detach().to(device).to(torch.float32)

model.eval()
generated_motion = []
with torch.no_grad():
    for (pose, _) in zip(model.stream(start_motion.unsqueeze(0).to(device)), range(block_size * 10)):
        generated_motion.append(pose)
generated_motion = torch.cat(generated_motion)

In [25]:
b = motion.frames[0]['root']
real_generated_frames = tensor_to_motion_frames(torch.cat([start_motion, generated_motion], dim=0).to(torch.double), [0.0, 0.0, 0.0], b[3:])

In [51]:
b = motion.frames[0]['root']
m = tensor_to_motion_frames(motion_tensor, [0, 10, 0], b[3:])
view = Viewer(skeleton,  motion.frames)   
view.run()